## Iteractively refine dataset labels

In [1]:
import os
import pathlib
import typing as t
import random
import requests
import json as json_package
import collections
import glob

import IPython.display
import ipywidgets
import datasets
import tokenizers
import ipywidgets
import colorama
import tqdm

import segmentador
from config import *
import interactive_labeling


%load_ext autoreload
%autoreload 2


CN = colorama.Fore.RED
CT = colorama.Fore.YELLOW
CLN = colorama.Fore.CYAN
CSR = colorama.Style.RESET_ALL


VOCAB_SIZE = 6000
AUTOSAVE_IN_N_INSTANCES = 20


DATASET_SPLIT = "test"
CACHED_INDEX_FILENAME = "refined_indices.csv"
BRUTE_DATASET_DIR = "../data"

TARGET_DATASET_NAME = f"df_tokenized_split_0_120000_{VOCAB_SIZE}"

REFINED_DATASET_DIR = os.path.join(BRUTE_DATASET_DIR, "refined_datasets", TARGET_DATASET_NAME)

BRUTE_DATASET_URI = os.path.join(BRUTE_DATASET_DIR, TARGET_DATASET_NAME)
CACHED_INDEX_URI = os.path.join(REFINED_DATASET_DIR, CACHED_INDEX_FILENAME)


assert BRUTE_DATASET_URI != REFINED_DATASET_DIR


logit_model = segmentador.BERTSegmenter(
    uri_model=f"../pretrained_segmenter_model/4_{VOCAB_SIZE}_layer_model",
    device="cpu",
)

pbar_dump = tqdm.auto.tqdm(desc="Instances until next dump:", total=AUTOSAVE_IN_N_INSTANCES)
it_counter = 0

random.seed(17)

Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]

In [2]:
tokenizer = tokenizers.Tokenizer.from_file(f"../tokenizers/{VOCAB_SIZE}_subwords/tokenizer.json")
tokenizer.get_vocab_size()

6000

## Load dataset

In [3]:
try:    
    with open(CACHED_INDEX_URI, "r") as f_index:
        cached_indices = set(map(int, f_index.read().split(",")))
        
    print(f"Loaded {len(cached_indices)} indices from disk.")

except FileNotFoundError:
    cached_indices = set()
    
    
new_refined_instances = collections.defaultdict(list)
df_brute = datasets.load_from_disk(BRUTE_DATASET_URI)
df_brute

Loaded 233 indices from disk.


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 114884
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 14361
    })
})

## Iteractive refinement

In [289]:
df_split = df_brute[DATASET_SPLIT]

fn_rand = lambda: random.randint(0, df_split.num_rows)

df_view_labels = df_split["labels"]
df_view_input_ids = df_split["input_ids"]

cls2id = {
    "no-op": 0,
    "seg": 1,
    "n-start": 2,
    "n-end": 3,
}

def print_labels(input_ids, labels, input_is_tokens: bool = False):
    seg_counter = 1
    
    print(end=CSR)
    print(end=f"{CLN}{seg_counter}.{CSR} ")
    
    if not input_is_tokens:
        tokens = list(map(tokenizer.id_to_token, input_ids))
    
    else:
        tokens = input_ids
    
    for i, (tok, lab) in enumerate(zip(tokens, labels)):
        if lab == cls2id["seg"]:
            seg_counter += 1
            print("\n\n", end=f"{CLN}{seg_counter}.{CSR} ")
        
        if lab == cls2id["n-start"]:
            print(end=CN)
            
        if lab == cls2id["n-end"]:
            print(end=CSR)
            
        print(tok, end=" ")
        
    return tokens


def dump_refined_dataset():
    new_subset = datasets.Dataset.from_dict(new_refined_instances, split=DATASET_SPLIT)
    shard_id = len(glob.glob(os.path.join(REFINED_DATASET_DIR, f"{DATASET_SPLIT}_*")))
    REFINED_DATASET_SHARD_URI = os.path.join(REFINED_DATASET_DIR, f"{DATASET_SPLIT}_{shard_id}")
    new_subset.save_to_disk(REFINED_DATASET_SHARD_URI)
    new_refined_instances.clear()
    
    with open(CACHED_INDEX_URI, "w") as f_index:
        f_index.write(",".join(map(str, sorted(cached_indices))))
    
    it_counter = 0
    
    print(f"Saved progress in '{REFINED_DATASET_SHARD_URI}'.")
    
    
def fn_run_cell(index_shift: int):
    js = IPython.display.Javascript(
        f"Jupyter.notebook.execute_cells([IPython.notebook.get_selected_index()+{index_shift}])"
    )
    IPython.display.display(js)

    
def fn_run_last_cell(_):
    js = IPython.display.Javascript(
        "Jupyter.notebook.execute_cells([IPython.notebook.ncells() - 1])"
    )
    IPython.display.display(js)

    
button_run_cell = ipywidgets.Button(
    description="Fetch new instance",
    tooltip="Run cell below to fetch a random instance",
    layout=ipywidgets.Layout(width="25%", height="48px", margin="0 0 0 5%"),
)
button_run_cell.on_click(lambda _: fn_run_cell(1))

button_run_cell_b = ipywidgets.Button(
    description="Fetch refined instance",
    tooltip="Fetch refined instance from front-end (run cell below)",
    style=dict(button_color="lightgreen")
)
button_run_cell_b.layout = button_run_cell.layout
button_run_cell_b.on_click(lambda _: fn_run_cell(1))

button_edit_instance = ipywidgets.Button(
    description="Edit instance",
    tooltip="Send instance to interactive front-end for refinement",
    style=dict(button_color="lightblue"),
)
button_edit_instance.layout = button_run_cell.layout
button_edit_instance.on_click(lambda _: fn_run_cell(2))

button_save = ipywidgets.Button(
    description="Save test instance",
    style=dict(button_color="salmon"),
    tooltip="Run notebook last cell (triggers code to save instance in curated dataset)",
)
button_save.layout = button_run_cell.layout
button_save.on_click(fn_run_last_cell)

In [195]:
IPython.display.display(ipywidgets.HBox((button_run_cell, button_edit_instance, button_save)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [904]:
id_ = fn_rand()
while id_ in cached_indices:
    id_ = fn_rand()


input_ids = df_view_input_ids[id_]
labels = df_view_labels[id_]
tokens = print_labels(input_ids, labels)

1. [CLS] REQUERIMENTO ( Do Sr . Ar ##ma ##ndo Ab ##ílio ) 

2. Requer o envio de Indicação ao Poder Executivo , com vistas à criação de Instituto Federal de Educação , Ciência e Tecnologia no Município de Sol ##ân ##e ##a , Estado da Paraíba . 

3. Senhor Presidente : Nos termos do art . 113 , inciso I e § 1º , do Regimento Interno da Câmara dos Deputados , requeiro a V . Exa . seja encaminhada ao Poder Executivo a Indicação anexa , sugerindo a criação de Instituto Federal de Educação , Ciência e Tecnologia no Município de Sol ##ân ##e ##a , Estado da Paraíba . 

4. Sala das Sessões , em de de 2009 . Deputado Ar ##ma ##ndo Ab ##ílio 2009 _ 87 ##84 

5. INDICAÇÃO No , DE 2009 ( Do Sr . Ar ##ma ##ndo Ab ##ílio ) 

6. Sugere ao Ministro de Estado da Educação a criação de Instituto Federal de Educação , Ciência e Tecnologia no Município de Sol ##ân ##e ##a , Estado da Paraíba . 

7. Excelentíssimo Senhor Ministro de Estado da Educação : A Lei n . º 11 . 89 ##2 , de 2008 , que institui a Re

In [897]:
logits = logit_model(df_split[id_], return_logits=True).logits
interactive_labeling.open_example(tokens, labels, logits=logits)
lock_save = True

In [292]:
IPython.display.display(ipywidgets.HBox((button_run_cell_b, button_save)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [898]:
ret = interactive_labeling.retrieve_refined_example()
labels = ret["labels"]
print_labels(tokens, labels, input_is_tokens=True)
lock_save = False

1. [CLS] INDICAÇÃO No , DE 2012 ( Do Sr . L ##E ##ON ##ARDO G ##AD ##EL ##HA ) 

2. Sugere o envio de Indicação ao Ministério da Fazenda , no sentido de se envi ##dar esforços perante o Banco do Brasil com vistas à instalação de uma agência no município de B ##om Su ##ce ##s ##so , no Estado da Paraíba . 

3. Excelentíssimo Senhor Ministro de Estado da Fazenda : Le ##va ##ndo - se em conta , pre ##l ##im ##inar ##mente , que o município de B ##om Su ##ce ##s ##so , tem uma população estabelecida , segundo levant ##amento de dados do último se ##ns ##o do IBGE , de 5 . 03 ##5 habitantes , sugerimos o emp ##enho desse respei ##tado órgão no sentido de envi ##dar esforços para inclusão da municipal ##idade no ro ##l de expansão das agências do Banco do Brasil . A proposta em ep ##íg ##ra ##f ##e é pertin ##ente uma vez que B ##om Su ##ce ##s ##so em termos popul ##acional é maior que Ca ##tu ##ri ##té ( estima ##da em 4 . 53 ##7 habitantes ) , já contempl ##ada com uma agência do Banco do

In [905]:
if not lock_save and id_ not in cached_indices:
    it_counter += 1
    cached_indices.add(id_)

    for key, val in df_split[id_].items():
        if key != "labels":
            new_refined_instances[key].append(val.copy())

        else:
            new_refined_instances[key].append(labels)
     
    pbar_dump.update()


if it_counter % AUTOSAVE_IN_N_INSTANCES == 0:
    dump_refined_dataset()
    pbar_dump.reset()
    
    
print(pbar_dump)

Saved progress in '../data/refined_datasets/df_tokenized_split_0_120000_6000/test_21'.
Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]
